<div style="background-color:rgb(245, 184, 242); color: rgb(70, 11, 83); border: 2px solidrgb(255, 255, 255); padding: 10px; border-radius: 7px;">
Notes on Utility

With only one bacteria, your only need 1 iteration
1 bacteria in iiFBA == FBA aka you only need one iteration

When you have one organism you will converge in one iteration, which is why is collapses to FBA

No need for time and rates because iterative convergence helps lead to the exact behavior we can expect in the first instance. 
^^ As long as growth is limited only by environmental flux and not internal constraints

If alpha zero, no insight to interaction unless there is cross-feeding, in which the consumer will consume after the first iteration
</div>

<div class= "alert alert-block alert-success">
My future steps: <br>
- Plotting sampling (unimportant) <br>
<br>

pip installable!!! (will add to pyPI when done-done)<br>
pip install "git+https://github.com/m-3-lab/iiFBA.git@main#subdirectory=package"

</div>

In [ ]:
import iifba
from iifba.utils import load_simple_models, load_example_models
import matplotlib.pyplot as plt
import numpy as np

models, media = load_simple_models(4)
EC, BT, media = load_example_models()

rel_abund = np.ones(len(models))  # equal relative abundance for all models
comm = iifba.Community([EC, BT], media, rel_abund=rel_abund, iters=10, objective="fba")
f, F = comm.iifba(early_stop=True) # defualt values

summ = comm.summary()
#comm.env_fluxes

print(summ)  # print summary to console
summ

In [ ]:
import iifba
from iifba.utils import load_simple_models
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
for sim_idx in [1,2,3, 4, 5, 6, 7]:
	print("\n", sim_idx)
	models, media = load_simple_models(sim_idx)

	rel_abund = np.ones(len(models))  # equal relative abundance for all models
	rel_abund[-1] = 1 * rel_abund[0]  # make last model more abundant
	comm = iifba.Community(models, media, rel_abund=rel_abund, iters=5)
	f, F = comm.iifba(early_stop=True, v=True) # defualt values

	biomass_id = "exchangeBio1"
	ax1 = iifba.iifba_vis(F.loc[0,:][biomass_id], ax=ax1, line_lab=sim_idx)
	if sim_idx > 2:
		biomass_id = "exchangeBio2"
		ax2 = iifba.iifba_vis(F.loc[1,:][biomass_id], ax=ax2, line_lab=sim_idx)
	else:
		ax2 = iifba.iifba_vis(np.zeros(5), ax=ax2, line_lab=sim_idx)
#adjust plot methods
for line_idx, line in enumerate(ax1.get_lines()):
	line.set_alpha(0.75)  # Set transparency for all lines in ax1
	line.set_lw(2*(8-line_idx))  # Vary line width for each line
for line_idx, line in enumerate(ax2.get_lines()):
	line.set_alpha(0.75)  # Set transparency for all lines in ax2
	line.set_lw(2*(8-line_idx))
# Update legends
ax1.legend(['Sim. 1', 'Sim. 2', 'Sim. 3', 'Sim. 4', 'Sim. 5', "Sim. 6", "Sim. 7"])
ax2.legend(['Sim. 1 (N/A)', 'Sim. 2 (N/A)', 'Sim. 3', 'Sim. 4', 'Sim. 5', "Sim. 6", "Sim. 7"])
# Method 3: Get and modify legend properties
legend1 = ax1.get_legend()
legend2 = ax2.get_legend()
# Use display() to show the updated plot
ax1.set_title("Biomass 1")
ax2.set_title("Biomass 2")
fig.tight_layout()
fig.suptitle(r"iiFBA Results for Various Simulations")


# Simple Models Implementation iiFBA


In [ ]:
import iifba
from iifba.utils import load_simple_models
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)
for sit_idx in [1,2,3, 4, 5, 6]:
	print("\n", sit_idx)
	models, media = load_simple_models(sit_idx)

	rel_abund = np.ones(len(models))  # equal relative abundance for all models
	rel_abund[-1] = 4 * rel_abund[0]  # make last model more abundant
	f, F = iifba.iipfba(models, media, rel_abund=rel_abund, iters=10) # defualt values

	biomass_id = "exchangeBio1"
	ax1 = iifba.iifba_vis(F.loc[0,:][biomass_id], ax=ax1, line_lab=sit_idx)
	if sit_idx > 2:
		biomass_id = "exchangeBio2"
		ax2 = iifba.iifba_vis(F.loc[1,:][biomass_id], ax=ax2, line_lab=sit_idx)
	else:
		ax2 = iifba.iifba_vis(np.zeros(10), ax=ax2, line_lab=sit_idx)
#adjust plot meh
for line_idx, line in enumerate(ax1.get_lines()):
	line.set_alpha(0.75)  # Set transparency for all lines in ax1
	line.set_lw(2*(7-line_idx))  # Vary line width for each line
for line_idx, line in enumerate(ax2.get_lines()):
	line.set_alpha(0.75)  # Set transparency for all lines in ax2
	line.set_lw(2*(7-line_idx))
# Update legends
ax1.legend(['Situation 1', 'Situation 2', 'Situation 3', 'Situation 4', 'Situation 5', "Situation 6"])
ax2.legend(['Situation 1 (N/A)', 'Situation 2 (N/A)', 'Situation 3', 'Situation 4', 'Situation 5', "Situation 6"])
# Method 3: Get and modify legend properties
legend1 = ax1.get_legend()
legend2 = ax2.get_legend()
# Use display() to show the updated plot
ax1.set_title("Biomass 1")
ax2.set_title("Biomass 2")
fig.tight_layout()
fig.suptitle(r"iiFBA Results for Various Situations")


In [ ]:
from IPython.display import display
for line in ax1.get_lines():
	line.set_alpha(0.75)  # Set transparency for all lines in ax1
	line.set_lw(2)
for line in ax2.get_lines():
	line.set_alpha(0.75)  # Set transparency for all lines in ax1
	line.set_lw(2)

ax1.get_lines()[2].set_lw(4)# 5 overlaps with 3
ax1.get_lines()[2].set_ls("--")

ax2.get_lines()[3].set_lw(4) # 5 overlaps with 4
ax2.get_lines()[3].set_ls("--")
ax2.get_lines()[2].set_lw(4) # 6 overlaps with 3
ax2.get_lines()[2].set_ls("--")
ax2.get_lines()[0].set_lw(0)
ax2.get_lines()[1].set_lw(0)

# Update legends
ax1.legend(['Situation 1', 'Situation 2', 'Situation 3', 'Situation 4', 'Situation 5', "Situation 6"])
ax2.legend(['Situation 1 (N/A)', 'Situation 2 (N/A)', 'Situation 3', 'Situation 4', 'Situation 5', "Situation 6"])

# Method 3: Get and modify legend properties
legend1 = ax1.get_legend()
legend2 = ax2.get_legend()

# Use display() to show the updated plot
ax1.set_title("Biomass 1")
ax2.set_title("Biomass 2")


fig.tight_layout()
fig.suptitle(r"iiFBA Results for Various Situations")
display(fig)


## Compartmentalized FBA 
Making compartmentalized FBA models based on each of the 9 simulations.

In [1]:
from cobra import Model
import pandas as pd
import cobra as cb
import iifba

cfba_results = pd.DataFrame(columns=["simulation", "Org1_growth", "Org2_growth"])

for sit_idx in ["1c", "2a", "2b","2c", "2d","2e", "3a", "3b", "3c"]: 
	models, media = iifba.utils.load_simple_models(sit_idx)
	print(f"\nSimulation {sit_idx} Models:")
	for model in models:
		for ex in model.exchanges:
			ex.lower_bound = 0  # Set lower bound to 0 for all exchange reactions
		for med_ex in media.keys():
			if med_ex in model.reactions:
				model.reactions.get_by_id(med_ex).lower_bound = media[med_ex]
				

	for model_idx, model in enumerate(models):
		# Change compartments for each model so that models[i] uses e0 and c{i+1} compartments
		for met in model.metabolites:
			if met.compartment == "c":
				met.compartment = f"c{model_idx+1}"
				id = met.id.replace("bio1", "bio_c").replace("bio2", "bio_c").replace("_c", f"_c{model_idx+1}")
				met.id = id
		
		for rxn in model.reactions:
			# Change the compartment of the reaction to match the new metabolite compartments
			for met in rxn.metabolites.copy():
				if met.compartment.startswith("c"):
					if not rxn.id.endswith(f"_{met.compartment}"):
					# Find the metabolite object with the new id in the model
						rxn.id = rxn.id + f"_{met.compartment}"

	# Merge the two models into a single compartmentalized model
	comp_model = Model(f"compartmentalized_sim{sit_idx}")
	for m in models:
		comp_model.add_metabolites([met.copy() for met in m.metabolites])
		comp_model.add_reactions([rxn.copy() for rxn in m.reactions])


	# Example: print the objective value after optimizing the compartmentalized model
	objective_reactions = [comp_model.reactions.get_by_id(f"exchangeBio{i+1}") for i in range(len(models))]
	objective_rxns_coef = [1] * len(objective_reactions)

			
	comp_model.objective = dict(zip(objective_reactions, objective_rxns_coef))

	solution = comp_model.optimize()
	cb.io.save_json_model(comp_model, f"Simple_Models/cFBA_Models/cFBA_sim{sit_idx}.json")
	print(f"{comp_model.id}, Objective value: {solution.objective_value}")
	print(solution.fluxes)

	cfba_results.loc[len(cfba_results)] = [sit_idx] + [solution.fluxes.get(f"exchangeBio{i+1}") for i in range(len(models))]

cfba_results.to_csv("Simple_Models/Results/cFBA_results.csv", index=False)

Set parameter Username
Set parameter LicenseID to value 2684253
Academic license - for non-commercial use only - expires 2026-07-02


Ignoring reaction 'Ex_A' since it already exists.
Ignoring reaction 'Ex_A' since it already exists.
Ignoring reaction 'Ex_A' since it already exists.
Ignoring reaction 'Ex_A' since it already exists.
Ignoring reaction 'Ex_A' since it already exists.
Ignoring reaction 'Ex_B' since it already exists.
Ignoring reaction 'Ex_A' since it already exists.
Ignoring reaction 'Ex_B' since it already exists.
Ignoring reaction 'Ex_C' since it already exists.
Ignoring reaction 'Ex_D' since it already exists.
Ignoring reaction 'Ex_A' since it already exists.
Ignoring reaction 'Ex_B' since it already exists.



Simulation 1c Models:
compartmentalized_sim1c, Objective value: 20.0
Ex_A             -10.0
T_A_c1            10.0
T_Bio1_c1         10.0
BiomassOrg1_c1    10.0
exchangeBio1      10.0
Ex_B             -10.0
T_B_c2            10.0
T_Bio2_c2         10.0
BiomassOrg2_c2    10.0
exchangeBio2      10.0
Name: fluxes, dtype: float64

Simulation 2a Models:
compartmentalized_sim2a, Objective value: 10.0
Ex_A             -10.0
T_A_c1             0.0
T_Bio1_c1          0.0
BiomassOrg1_c1     0.0
exchangeBio1       0.0
T_A_c2            10.0
T_Bio2_c2         10.0
BiomassOrg2_c2    10.0
exchangeBio2      10.0
Name: fluxes, dtype: float64

Simulation 2b Models:
compartmentalized_sim2b, Objective value: 10.0
Ex_A             -10.0
T_A_c1             0.0
T_Bio1_c1          0.0
BiomassOrg1_c1     0.0
exchangeBio1       0.0
T_A_c2            10.0
T_Bio2_c2         10.0
BiomassOrg2_c2    10.0
exchangeBio2      10.0
Name: fluxes, dtype: float64

Simulation 2c Models:
compartmentalized_sim2c, Objective v



| Simulation | cFBA Biomass 1 | cFBA Biomass 2|
|:----:|:-------:|:---:|
|1a| 10| N/A|
|1b|20| N/A|
|1c|10|10|
|2a|0<=B1<=10| B2=10-B1|
|2b|0<=B1<=10| B2=10-B1|
|2c| 2| 8|
|3a|20 |10 |
|3c|20|20|
|3b|10|10|


## MICOM


In [2]:
import micom
import pandas as pd
import iifba 
from micom import Community

micom_results = pd.DataFrame(columns=["simulation", "tradeoff", "Org1_growth", "Org2_growth"])

for sit_idx in ["1c","2a","2b", "2c", "2d", "2e", "3a", "3b", "3c"]: # "2a", "2b","3a", "3b", "3c"
	# get media from iiFBA
	_ , media = iifba.utils.load_simple_models(sit_idx)
	media = {k.upper() + '_m': -v for k, v in media.items()}
	
	# make sure 2b (same model, different abundances) uses correct abundances
	abund = [0.2, 0.8] if sit_idx == "2b" else [0.5, 0.5]
	sim_idx = sit_idx if sit_idx != "2b" else "2a"

	# create community dataframe
	community = pd.DataFrame({
		"id": ["Org1", "Org2"],
		"file": [f"Simple_Models/Cobra_Models/sim{sim_idx}_1.json", f"Simple_Models/Cobra_Models/sim{sim_idx}_2.json"],
		"abundance": abund
	})

	# create micom community
	community = Community(community)
	community.medium = media
	community.solver = "gurobi"

	print(sit_idx)
	solutions = community.cooperative_tradeoff(fraction=[1, 0.8, 0.6, 0.4, 0.2, 0], pfba=True)
	for idx, sol in enumerate(solutions["solution"]):
		print(f"Solution alpha: {solutions['tradeoff'][idx]}", end="  ")
		# print(sol.members)
		org_growth = (sol.members["abundance"] * sol.members["growth_rate"]).to_numpy()[0:-1]
		print(org_growth)
		micom_results.loc[len(micom_results)] = [sit_idx, solutions['tradeoff'][idx], org_growth[0], org_growth[1]]

		
	print()

micom_results.to_csv("Simple_Models/Results/micom_results.csv", index=False)


Output()

[10/22/25 15:02:46] WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=401643;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=805835;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

Output()

1c
Solution alpha: 1.0  [10. 10.]
Solution alpha: 0.8  [8. 8.]
Solution alpha: 0.6  [6. 6.]
Solution alpha: 0.4  [4. 4.]
Solution alpha: 0.2  [2. 2.]
Solution alpha: 0.0  [0. 0.]



                    WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=189509;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=332795;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

...

                    WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=229332;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=528684;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

3b
Solution alpha: 1.0  [20. 20.]
Solution alpha: 0.8  [16.000015 15.999985]
Solution alpha: 0.6  [12.00001707 11.99998293]
Solution alpha: 0.4  [8.00002203 7.99997797]
Solution alpha: 0.2  [4.00003 3.99997]
Solution alpha: 0.0  [5.75027057e-06 3.45254828e-06]



Output()

                    WARNING  There does not seem to be any carbon source in your medium. Please    ]8;id=457821;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py\community.py]8;;\:]8;id=310445;file:///home/rseag/anaconda3/envs/MBE2/lib/python3.10/site-packages/micom/community.py#667\667]8;;\
                             double-check your medium IDs.                                                         

3c
Solution alpha: 1.0  [10. 10.]
Solution alpha: 0.8  [8. 8.]
Solution alpha: 0.6  [6. 6.]
Solution alpha: 0.4  [4. 4.]
Solution alpha: 0.2  [2. 2.]
Solution alpha: 0.0  [4.69546032e-06 5.13516786e-06]





| Simulation | alpha | MICOM Biomass 1 | MICOM Biomass 2|
|:----:|:-------:|:-------:|:---:|
|1a|                |                |               |
|1b|                |                |               |
|1c|         0      |     0           |     0           |
|1c|        .2      |     2           |     2           |
|1c|        .4      |     4           |     4           |
|1c|        .6      |     6           |     6           |
|1c|         .8     |     8           |     8           |
|1c|          1     |     10          |     10          |
|-------------|-------------|-------------|-------------|
|2a|         0      |    0            |      0         |
|2a|        .2      |    1            |      1         |
|2a|        .4      |    2            |      2         |
|2a|        .6      |    3            |      3         |
|2a|         .8     |    4            |      4         |
|2a|          1     |    5            |      5         |
|-------------|-------------|-------------|-------------|
|2b|         0      |    0           |      0        |
|2b|        .2      |   0.11764706   |  1.88235294   |
|2b|        .4      |   0.23529412   |  3.76470588   |
|2b|        .6      |   0.35294118   |  5.64705882   |
|2b|         .8     |   0.47058824   |  7.52941176   |
|2b|          1     |   0.58823529   |  9.41176471   |
|-------------|-------------|-------------|-------------|
|2c|         0      |    0            |      0         |
|2c|        .2      |    0.5          |      0.5       |
|2c|        .4      |    1            |      1         |
|2c|        .6      |    2            |      1         |
|2c|         .8     |    3            |      1         |
|2c|          1     |    4            |      1         |
|-------------|-------------|-------------|-------------|
|3a|         0      |     0           |       0       |
|3a|        .2      |     3           |       3       |
|3a|        .4      |     6           |       6       |
|3a|        .6      |     9           |       9       |
|3a|         .8     |    14           |      10       |
|3a|          1     |    20           |      10       |
|-------------|-------------|-------------|-------------|
|3b|         0      |    0            |      0          |
|3b|        .2      |    4            |      4          |
|3b|        .4      |    8            |      8          |
|3b|        .6      |    12           |      12         |
|3b|         .8     |    16           |      16         |
|3b|          1     |    20           |      20         |
|-------------|-------------|-------------|-------------|
|3c|         0      |    0            |      0         |
|3c|        .2      |    2            |      2         |
|3c|        .4      |    4            |      4         |
|3c|        .6      |    6            |      6         |
|3c|         .8     |    8            |      8         |
|3c|          1     |   10            |      10        |


In [ ]:
import iifba
from iifba.utils import load_simple_models
import matplotlib.pyplot as plt
import numpy as np

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6), sharey=True)

models, media = load_simple_models(3)

f, F = iifba.iipfba(models, media, rel_abund=[0.2,0.8], iters=10) # defualt values

biomass_id = "exchangeBio1"
ax1 = iifba.iifba_vis(F.loc[0,:][biomass_id], ax=ax1, line_lab=sit_idx)

biomass_id = "exchangeBio2"
ax2 = iifba.iifba_vis(F.loc[1,:][biomass_id], ax=ax2, line_lab=sit_idx)


In [ ]:
import cobra

# Create a new model
model = cobra.Model("toy_multicompartment")

# Define compartments
model.compartments = {"c": "cytosol", "e": "extracellular"}

# Define metabolites (simple letter names)
A_e = cobra.Metabolite("A_e", compartment="e")
A_c = cobra.Metabolite("A_c", compartment="c")
B_c = cobra.Metabolite("B_c", compartment="c")
C_e = cobra.Metabolite("C_e", compartment="e")
C_c = cobra.Metabolite("C_c", compartment="c")

# Exchange with environment (A import, C export)
EX_A_e = cobra.Reaction("EX_A_e")
EX_A_e.add_metabolites({A_e: -1})
EX_A_e.lower_bound = -1000  # allow uptake
EX_A_e.upper_bound = 1000

EX_C_e = cobra.Reaction("EX_C_e")
EX_C_e.add_metabolites({C_e: -1})
EX_C_e.lower_bound = -1000    # allow secretion
EX_C_e.upper_bound = 1000

# Transport between compartments
TRANS_A = cobra.Reaction("TRANS_A")
TRANS_A.add_metabolites({A_e: -1, A_c: 1})
TRANS_A.lower_bound = -1000
TRANS_A.upper_bound = 1000

TRANS_C = cobra.Reaction("TRANS_C")
TRANS_C.add_metabolites({C_c: -1, C_e: 1})
TRANS_C.lower_bound = -1000
TRANS_C.upper_bound = 1000

# Internal cytosolic reactions
R1 = cobra.Reaction("R1")
R1.add_metabolites({A_c: -1, B_c: 1})

R2 = cobra.Reaction("R2")
R2.add_metabolites({B_c: -1, C_c: 1})

# Add reactions to the model
model.add_reactions([EX_A_e, EX_C_e, TRANS_A, TRANS_C, R1, R2])

# Set objective (e.g., maximize C export)
model.objective = "R2"

# Test FBA
sol = model.optimize()
print(f"FBA status: {sol.status}, Objective value: {sol.objective_value}")

# Optional: Test sampling (may require mor2:e degrees of freedom in tiny models)
try:
    samples = cobra.sampling.sample(model, n=10)
    print(samples.head())
except Exception as e:
    print(f"Sampling failed: {e}")

In [ ]:
import iifba
from iifba.utils import load_simple_models, load_example_models, plot_sampling
import matplotlib.pyplot as plt
import numpy as np

# EC, BT, media = load_example_models()
# models = [EC, BT]
# models, media = load_simple_models(5)
import copy
models = [copy.deepcopy(model), copy.deepcopy(model)]
media = {"EX_A_e":-10, "EX_C_e": -10}

rel_abund = np.ones(len(models))  # equal relative abundance for all models
comm = iifba.Community(models, media, rel_abund=rel_abund, iters=10, objective="fba")
comm.iifba_sampling(m_vals=[10,10], convergence=None, v=True, objective_percent=0.2) # defualt values

comm.env_fluxes
plot_sampling(comm, model_idx=0)
# summ = comm.summary()
# #comm.env_fluxes

# print(summ)  # print summary to console
# summ

In [ ]:
comm.cumulative_fluxes

In [ ]:
org = comm.org_fluxes
cum = comm.cumulative_fluxes
M = comm.M
